In [1]:
import argparse
import glob
import os
import time

import numpy as np
import pandas as pd
import cv2

In [2]:
parser = {
    'val_path': '../data/val768/',
    'val_masks_path': '../data/val768_masks/',
    'test_path': '../data/test_hq/',
    'intermediate_path': '../intermediate/output/',
    'output_paths': ['output_tn_41/'],
    'submission_name': 'subu45.csv',
    'log_every': 1000,
}
args = argparse.Namespace(**parser)

args.output_paths = [os.path.join(args.intermediate_path, x)
                     for x in args.output_paths]
args.img_names = sorted([x.split('/')[-1]
                         for x in glob.glob(args.test_path + '/*.jpg')])

In [3]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of 
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask, 
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [ ]:
def get_mask(output_paths, index, weights=None):
    if weights is None:
        weights = [1] * len(output_paths)
    
    img = 0
    for i, path in enumerate(output_paths):
        file = os.path.join(path, str(index)+'.jpg')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
        img = img + image * weights[i]
    img = img / sum(weights)
    return img > 127.5

In [ ]:
list_rle = []
init_time = time.time()
for i in range(1, len(args.img_names)+1):
    mask = get_mask(args.output_paths, i)
    rle = rle_encode(mask)
    list_rle.append(' '.join(str(x) for x in rle))
    
    if i % args.log_every == 0:
        print("Done {:6} images in {:.0f}s".format(i, time.time()-init_time))

Done   1000 images in 22s
Done   2000 images in 44s
Done   3000 images in 67s
Done   4000 images in 88s
Done   5000 images in 110s
Done   6000 images in 132s
Done   7000 images in 154s
Done   8000 images in 177s
Done   9000 images in 199s
Done  10000 images in 224s
Done  11000 images in 248s
Done  12000 images in 270s
Done  13000 images in 292s
Done  14000 images in 314s
Done  15000 images in 336s
Done  16000 images in 358s
Done  17000 images in 380s
Done  18000 images in 403s
Done  19000 images in 426s
Done  20000 images in 448s
Done  21000 images in 470s
Done  22000 images in 491s
Done  23000 images in 514s
Done  24000 images in 536s
Done  25000 images in 558s
Done  26000 images in 580s
Done  27000 images in 601s
Done  28000 images in 624s
Done  29000 images in 647s
Done  30000 images in 669s
Done  31000 images in 691s
Done  32000 images in 714s
Done  33000 images in 735s
Done  34000 images in 757s
Done  35000 images in 779s
Done  36000 images in 803s
Done  37000 images in 826s
Done 

In [ ]:
df = pd.DataFrame({"img": args.img_names, "rle_mask": list_rle})
submiss_path = os.path.join(args.intermediate_path, args.submission_name)
df.to_csv(submiss_path, index=False)